Urban Data Science & Smart Cities <br>
URSP688Y Spring 2025<br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

# Exercise04

This last exercise is an opportunity for you to get started on your final project. Please identify a portion of your project to get started on and submit a notebook (and any other related files) where you:

1. State the question you are aiming to address with this portion of your analysis
2. Outline the approach you will use to answer that question (pseudocode or you can start to more formally outline the approach section for your final narrative)
3. Operationalize your approach with data and code that you can later slot into your final analysis

## Submitting

Please make a pull request with all of your code and reasonably-sized data in a folder with your first name. See the example with my name in the `exercise04` directory.

If you have datasets that are too large for GitHub or should not be made public, please upload them to a cloud location (e.g., Google Drive) to which I (and ideally your classmates) have access. Please also provide instructions for how someone running your code should properly locate or connect to these files so the analysis will run properly. For example, should they copy and paste the files into the same directory as your notebook, or a provided `data` directory? Best practice is to include these instructions in a separate ReadMe.md or ReadMe.txt file, or at the top of your notebook.

In [ ]:
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>Zoning Lookup by Class</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">

  <!-- Leaflet CSS & JS -->
  <link
    rel="stylesheet"
    href="https://unpkg.com/leaflet@1.9.3/dist/leaflet.css"
  />
  <script src="https://unpkg.com/leaflet@1.9.3/dist/leaflet.js"></script>

  <!-- Turf.js for point-in-polygon -->
  <script src="https://unpkg.com/@turf/turf@6/turf.min.js"></script>

  <style>
    html,body { margin:0; padding:0; height:100% }
    #map { width:100%; height:100% }
    .info {
      position: absolute;
      top: 10px; right: 10px;
      z-index: 1000;
      background: white;
      padding: 8px 12px;
      border-radius: 4px;
      box-shadow: 0 0 6px rgba(0,0,0,0.3);
      font-family: sans-serif;
    }
  </style>
</head>
<body>
  <div id="map"></div>
  <div class="info" id="zoneInfo">Loading zoning…</div>

  <script>
    // ←— replace this with your “Raw” GitHub URL:
    const ZONES_GEOJSON_URL = 
      'https://raw.githubusercontent.com/arazidavid/zon/main/zone.geojson';

    // Initialize Leaflet map
    const map = L.map('map').setView([38.98, -76.93], 13);
    L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
      attribution: '© OpenStreetMap contributors'
    }).addTo(map);

    // empty GeoJSON layer for your zones
    const zoneLayer = L.geoJSON(null, {
      style: f => ({ color: '#3388ff', weight: 2, fillOpacity: 0.2 })
    }).addTo(map);

    // fetch & draw your zones
    fetch(ZONES_GEOJSON_URL)
      .then(r => r.json())
      .then(data => {
        zoneLayer.addData(data);
        map.fitBounds(zoneLayer.getBounds());
        document.getElementById('zoneInfo').textContent =
          'Zones loaded. Waiting for your location…';
      })
      .catch(err => {
        console.error(err);
        zoneInfo.textContent = 'Failed to load zoning data.';
      });

    // geolocate & test point-in-polygon
    const info = document.getElementById('zoneInfo');
    if (!navigator.geolocation) {
      info.textContent = 'Geolocation not supported';
    } else {
      navigator.geolocation.getCurrentPosition(pos => {
        const [lon, lat] = [
          pos.coords.longitude,
          pos.coords.latitude
        ];
        // center map on you
        map.setView([lat, lon], 15);

        // turf point
        const pt = turf.point([lon, lat]);

        // look for a containing polygon by its CLASS field
        let foundClass = null;
        zoneLayer.eachLayer(layer => {
          if (!foundClass &&
              turf.booleanPointInPolygon(pt, layer.feature)
          ) {
            foundClass = layer.feature.properties.CLASS;
          }
        });

        // show result
        if (foundClass) {
          info.innerHTML = `✔️ You are inside zoning <b>${foundClass}</b>`;
        } else {
          info.innerHTML = `⚠️ You are <i>not</i> inside any zone`;
        }

        // drop a marker
        L.marker([lat, lon])
          .addTo(map)
          .bindPopup(info.textContent)
          .openPopup();

      }, err => {
        console.error(err);
        info.textContent = 'Unable to get your location';
      });
    }
  </script>
</body>
</html>

In [ ]:
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8" />
  <title>Smart Zoning Assistant</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />

  <!-- Leaflet CSS & JS -->
  <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.3/dist/leaflet.css" />
  <script src="https://unpkg.com/leaflet@1.9.3/dist/leaflet.js"></script>

  <!-- Turf.js -->
  <script src="https://unpkg.com/@turf/turf@6/turf.min.js"></script>

  <style>
    html, body {
      margin: 0;
      padding: 0;
      height: 100%;
      overflow: hidden;
    }
    #map {
      height: 100vh;
      width: 100%;
    }
    .info {
      position: absolute;
      top: 10px;
      right: 10px;
      z-index: 1000;
      background: white;
      padding: 10px 15px;
      border-radius: 5px;
      font-family: Arial, sans-serif;
      box-shadow: 0 2px 6px rgba(0, 0, 0, 0.2);
    }
    #searchInput {
      position: absolute;
      top: 10px;
      left: 10px;
      z-index: 1001;
      width: 90%;
      max-width: 400px;
      padding: 8px 12px;
      font-size: 16px;
      border: 1px solid #ccc;
      border-radius: 5px;
    }
  </style>
</head>
<body>

  <input id="searchInput" type="text" placeholder="Enter an address (e.g. 123 Main St)" />
  <div id="map"></div>
  <div class="info" id="zoneInfo">Loading zoning data...</div>

  <script>
    const ZONES_GEOJSON_URL = 'https://raw.githubusercontent.com/arazidavid/zon/main/zone.geojson';

    const map = L.map('map').setView([38.98, -76.93], 13);
    L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
      attribution: '© OpenStreetMap contributors'
    }).addTo(map);

    const zoneLayer = L.geoJSON(null, {
      style: feature => ({
        color: '#3388ff',
        weight: 2,
        fillOpacity: 0.2
      })
    }).addTo(map);

    const info = document.getElementById('zoneInfo');

    const zoneDescriptions = {
      "RR": "Rural Residential – Large-lot single-family homes",
      "R-55": "Single-Family Detached Residential – 6,500 sq ft min",
      "R-T": "Townhouses – Medium-density attached residential",
      "M-X-T": "Mixed-Use, Transportation Oriented",
      "C-S-C": "Commercial Shopping Center",
      "I-1": "Light Industrial",
      "O-S": "Open Space",
      // Add more codes as needed
    };

    fetch(ZONES_GEOJSON_URL)
      .then(res => res.json())
      .then(data => {
        zoneLayer.addData(data);
        info.textContent = 'Zoning data loaded. Awaiting your location...';
      })
      .catch(err => {
        console.error(err);
        info.textContent = '❌ Failed to load zoning data.';
      });

    function checkZoning(lat, lon, label = "Your location") {
      const pt = turf.point([lon, lat]);
      let foundClass = null;

      zoneLayer.eachLayer(layer => {
        const geom = layer.feature.geometry;
        if (!foundClass && turf.booleanPointInPolygon(pt, turf.feature(geom))) {
          foundClass = layer.feature.properties.CLASS;
        }
      });

      let description = zoneDescriptions[foundClass] || "Zone info not available";
      if (foundClass) {
        info.innerHTML = `📍 <b>${label}</b><br>✅ Zone: <b>${foundClass}</b><br><small>${description}</small>`;
      } else {
        info.innerHTML = `📍 <b>${label}</b><br>⚠️ Not inside a known zone.`;
      }

      L.marker([lat, lon])
        .addTo(map)
        .bindPopup(info.innerHTML)
        .openPopup();

      map.setView([lat, lon], 15);
    }

    // Geolocation
    if (navigator.geolocation) {
      navigator.geolocation.getCurrentPosition(
        pos => {
          checkZoning(pos.coords.latitude, pos.coords.longitude);
        },
        err => {
          console.error(err);
          info.textContent = '⚠️ Location access denied.';
        }
      );
    } else {
      info.textContent = '⚠️ Geolocation not supported.';
    }

    // Address search with Nominatim
    document.getElementById('searchInput').addEventListener('keypress', function(e) {
      if (e.key === 'Enter') {
        const query = this.value;
        fetch(`https://nominatim.openstreetmap.org/search?format=json&q=${encodeURIComponent(query)}`)
          .then(res => res.json())
          .then(data => {
            if (data && data.length > 0) {
              const lat = parseFloat(data[0].lat);
              const lon = parseFloat(data[0].lon);
              checkZoning(lat, lon, query);
            } else {
              info.textContent = 'Address not found.';
            }
          })
          .catch(err => {
            console.error(err);
            info.textContent = 'Search failed.';
          });
      }
    });
  </script>
</body>
</html>
